### Regression Analysis

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm

In [3]:
data = pd.read_csv("data/cereal_merged_df.csv")
data

,Unnamed: 0,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,...,PRODUCT_SIZE,STORE_ID,STORE_NAME,ADDRESS_CITY_NAME,ADDRESS_STATE_PROV_CODE,MSA_CODE,SEG_VALUE_NAME,PARKING_SPACE_QTY,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,0,2009-01-14,367.0,1111085319,14.0,13.0,13.0,26.32,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
1,1,2009-01-21,367.0,1111085319,12.0,12.0,12.0,22.68,1.89,1.89,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
2,2,2009-01-28,367.0,1111085319,18.0,17.0,16.0,33.66,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
3,3,2009-02-04,367.0,1111085319,13.0,13.0,13.0,24.44,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
4,4,2009-02-11,367.0,1111085319,16.0,16.0,16.0,29.92,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169684,169684,2011-12-07,29159.0,88491212971,12.0,12.0,12.0,34.20,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169685,169685,2011-12-14,29159.0,88491212971,14.0,14.0,14.0,39.90,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169686,169686,2011-12-21,29159.0,88491212971,22.0,22.0,21.0,62.70,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169687,169687,2011-12-28,29159.0,88491212971,17.0,16.0,16.0,48.45,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051


### Organize DataFrames for the analysis

In [7]:
data.MANUFACTURER.value_counts()

MANUFACTURER
GENERAL MI       35948
PRIVATE LABEL    35902
KELLOGG          35832
POST FOODS       31107
QUAKER           30900
Name: count, dtype: int64

In [9]:
data.SUB_CATEGORY.value_counts()

SUB_CATEGORY
ALL FAMILY CEREAL    82006
KIDS CEREAL          54690
ADULT CEREAL         32993
Name: count, dtype: int64

In [11]:
# create 2 DataFrames based on SUB_CATEGORY
family = data.loc[data['SUB_CATEGORY']=='ALL FAMILY CEREAL']
kids= data.loc[data['SUB_CATEGORY']=='KIDS CEREAL']

# store original mean price 
family_price = family.PRICE.mean()
kids_price = kids.PRICE.mean()

print('Family - mean price is:', family_price)
print('Kids - mean price is:', kids_price)

Family - mean price is: 2.8015646416115896
Kids - mean price is: 2.6833255924517267


In [13]:
# for preparing data for regression 
def data_prep(data):
    # kellog
    df = data.loc[data['MANUFACTURER']=="KELLOGG"].groupby(['WEEK_END_DATE','STORE_NUM']).agg({'PRICE':'mean','UNITS':'sum'}).reset_index()
    df.rename(columns={"PRICE":'kellog_price','UNITS':'kellog_units'},inplace=True)
    
    # other brands
    mi = data.loc[data['MANUFACTURER']=="GENERAL MI"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    private = data.loc[data['MANUFACTURER']=="PRIVATE LABEL"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    post = data.loc[data['MANUFACTURER']=="POST FOODS"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    quaker = data.loc[data['MANUFACTURER']=="QUAKER"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']

    df['mi_price'] = mi
    df['private_price'] = private
    df['post_price'] = post
    df['quaker_price'] = quaker
    
    # check # of transactions for each brand 
    print('kellog:',df.shape[0])
    print('General MI:',mi.shape[0])
    print('Private Label:',private.shape[0])
    print('Post Foods:',post.shape[0])
    print('Quarker:',quaker.shape[0])
    
    # convert prices variables to log-prices
    to_log = ['kellog_units','kellog_price','mi_price','private_price','post_price','quaker_price']
    df[to_log] = np.log(df[to_log])
    
    # specify categorical variables
    df['STORE_NUM'] = df['STORE_NUM'].astype('category')
    
    return (df)

### Family

In [16]:
df = data_prep(family) # post doesn't do Family cereal

kellog: 11904
General MI: 11987
Private Label: 11986
Post Foods: 0
Quarker: 10232


In [18]:
# get rid of rows with NaN
df = df.loc[df['quaker_price'].notnull()]
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 10232 entries, 0 to 10231
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   WEEK_END_DATE  10232 non-null  object  
 1   STORE_NUM      10232 non-null  category
 2   kellog_price   10232 non-null  float64 
 3   kellog_units   10232 non-null  float64 
 4   mi_price       10232 non-null  float64 
 5   private_price  10232 non-null  float64 
 6   post_price     0 non-null      float64 
 7   quaker_price   10232 non-null  float64 
dtypes: category(1), float64(6), object(1)
memory usage: 652.1+ KB


#### Model 1: only price variables 

In [21]:
Y=df['kellog_units']
X=df[['kellog_price','mi_price','private_price','quaker_price']]
X = sm.add_constant(X) # adding a constant

model_1 = sm.OLS(Y,X).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     219.0
Date:                Thu, 23 Jan 2025   Prob (F-statistic):          1.20e-180
Time:                        21:55:03   Log-Likelihood:                -11811.
No. Observations:               10232   AIC:                         2.363e+04
Df Residuals:                   10227   BIC:                         2.367e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.1087      0.144     28.589      0.000       3.827       4.390
kellog_price     -1.7979      0.064    -27.996      0.000      -1.924      -1.672
mi_price          0.1872      0.071      2.637      0.008       0.048       0.326
private_price     1.1652      0.128      9.108      0.000       0.914       1.416
quaker_price      0.1686      0.039      4.349      0.000       0.093       0.245
==============================================================================
Omnibus:                      961.732   Durbin-Watson:                   1.679
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1428.985
Skew:                          -0.726   Prob(JB):                    5.01e-311
Kurtosis:                       4.115   Cond. No.                         51.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-1.7979/(1-1.7979))
price

3.787642363068161

#### Model 2: price variables + store_num

In [26]:
# dummy variable
df = pd.get_dummies(data=df,columns=['STORE_NUM'], drop_first = True)

In [28]:
Y=df['kellog_units']
X = df.drop(['WEEK_END_DATE','kellog_units','post_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_2 = sm.OLS(Y,X).fit()
model_2.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [32]:
# Ensure only numeric columns in X and Y
X = df.drop(['WEEK_END_DATE', 'kellog_units', 'post_price'], axis=1)
X = X.select_dtypes(include=['number'])  # Keep only numeric columns

Y = df['kellog_units']

# Handle missing values (if any)
X = X.dropna()  # Optionally, drop rows with NaN in X
Y = Y[X.index]  # Ensure Y is aligned with X

# Add constant (intercept term)
X = sm.add_constant(X)

# Fit the OLS model
model_2 = sm.OLS(Y, X).fit()

# Display summary
print(model_2.summary())


                            OLS Regression Results                            
Dep. Variable:           kellog_units   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     219.0
Date:                Thu, 23 Jan 2025   Prob (F-statistic):          1.20e-180
Time:                        21:55:33   Log-Likelihood:                -11811.
No. Observations:               10232   AIC:                         2.363e+04
Df Residuals:                   10227   BIC:                         2.367e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.1087      0.144     28.589

In [34]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-2.4146/(1-2.4146))
price

2.86921729830426

#### Model 3: price variables + store_num + week_end_date 

In [37]:
# dummy variable
df = pd.get_dummies(data=df,columns=['WEEK_END_DATE'], drop_first = True)

In [39]:
Y=df['kellog_units']
X = df.drop(['kellog_units','post_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_3 = sm.OLS(Y,X).fit()
model_3.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [41]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-1.9090/(1-1.9090))
price

3.5301563701891245

#### Bias induced by not considering competition

In [44]:
# regression with No competitors
Y=df['kellog_units']
#X=df[['kellog_price']]
X = df.drop(['kellog_units','post_price','mi_price','private_price','quaker_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_1_b = sm.OLS(Y,X).fit()
model_1_b.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [46]:
X = df.drop(['kellog_units','post_price','mi_price','private_price','quaker_price'],axis=1)
X = X.select_dtypes(include=['number']).dropna()  # Keep only numeric columns and drop NaNs

Y = df['kellog_units']
Y = Y[X.index]  # Ensure Y is aligned with X

X = sm.add_constant(X)

model_1_b = sm.OLS(Y, X).fit()
model_1_b.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     764.1
Date:                Thu, 23 Jan 2025   Prob (F-statistic):          2.85e-162
Time:                        21:56:29   Log-Likelihood:                -11863.
No. Observations:               10232   AIC:                         2.373e+04
Df Residuals:                   10230   BIC:                         2.374e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            5.2355      0.072     72.374      0.000       5.094       5.377
kellog_price    -1.7718      0.064    -27.643      0.000      -1.897      -1.646
==============================================================================
Omnibus:                      950.133   Durbin-Watson:                   1.653
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1404.177
Skew:                          -0.721   Prob(JB):                    1.22e-305
Kurtosis:                       4.101   Cond. No.                         19.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [48]:
with_comp = price
#no_comp = (family_price*0.6)*(-1.7718/(1-1.7718))
no_comp = (family_price*0.6)*(-1.9079/(1-1.9079))
no_comp

3.5323968585069405

In [50]:
bias = with_comp - no_comp
percent_bias = 100*(bias/with_comp)
percent_bias

-0.06346711258277629

### Kids

In [53]:
df = data_prep(kids) # General MI and Private don't do Kids cereal

kellog: 11987
General MI: 0
Private Label: 0
Post Foods: 10094
Quarker: 10588


In [55]:
# get rid of rows with NaN
df = df.loc[df['post_price'].notnull()]
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 10094 entries, 0 to 10093
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   WEEK_END_DATE  10094 non-null  object  
 1   STORE_NUM      10094 non-null  category
 2   kellog_price   10094 non-null  float64 
 3   kellog_units   10094 non-null  float64 
 4   mi_price       0 non-null      float64 
 5   private_price  0 non-null      float64 
 6   post_price     10094 non-null  float64 
 7   quaker_price   10094 non-null  float64 
dtypes: category(1), float64(6), object(1)
memory usage: 643.4+ KB


#### Model 1: only price variables 

In [58]:
Y=df['kellog_units']
X=df[['kellog_price','post_price','quaker_price']]
X = sm.add_constant(X) # adding a constant

model_1 = sm.OLS(Y,X).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     2563.
Date:                Thu, 23 Jan 2025   Prob (F-statistic):               0.00
Time:                        21:56:34   Log-Likelihood:                -9040.4
No. Observations:               10094   AIC:                         1.809e+04
Df Residuals:                   10090   BIC:                         1.812e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            7.9302      0.074    107.494      0.000       7.786       8.075
kellog_price    -4.1487      0.048    -86.435      0.000      -4.243      -4.055
post_price       0.2728      0.039      7.045      0.000       0.197       0.349
quaker_price     0.2849      0.045      6.323      0.000       0.197       0.373
==============================================================================
Omnibus:                      160.092   Durbin-Watson:                   1.358
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.475
Skew:                          -0.142   Prob(JB):                     1.01e-57
Kurtosis:                       3.737   Cond. No.                         28.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [60]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-4.1487/(1-4.1487))
price

2.2147904650148953

#### Model 2: price variables + store_num

In [63]:
# dummy variable
df = pd.get_dummies(data=df,columns=['STORE_NUM'], drop_first = True)

In [65]:
Y=df['kellog_units']
X = df.drop(['WEEK_END_DATE','kellog_units','mi_price','private_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_2 = sm.OLS(Y,X).fit()
model_2.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [67]:
X = df.drop(['WEEK_END_DATE','kellog_units','mi_price','private_price'],axis=1)
X = X.select_dtypes(include=['number']).dropna()  # Keep only numeric columns and drop NaNs

Y = df['kellog_units']
Y = Y[X.index]  # Ensure Y is aligned with X

X = sm.add_constant(X)

model_2 = sm.OLS(Y, X).fit()
model_2.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     2563.
Date:                Thu, 23 Jan 2025   Prob (F-statistic):               0.00
Time:                        21:56:57   Log-Likelihood:                -9040.4
No. Observations:               10094   AIC:                         1.809e+04
Df Residuals:                   10090   BIC:                         1.812e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            7.9302      0.074    107.494      0.000       7.786       8.075
kellog_price    -4.1487      0.048    -86.435      0.000      -4.243      -4.055
post_price       0.2728      0.039      7.045      0.000       0.197       0.349
quaker_price     0.2849      0.045      6.323      0.000       0.197       0.373
==============================================================================
Omnibus:                      160.092   Durbin-Watson:                   1.358
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.475
Skew:                          -0.142   Prob(JB):                     1.01e-57
Kurtosis:                       3.737   Cond. No.                         28.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [69]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-4.1813/(1-4.1813))
price

2.209319882306706

#### Model 3: price variables + store_num + week_end_date 

In [72]:
# dummy variable
df = pd.get_dummies(data=df,columns=['WEEK_END_DATE'], drop_first = True)

In [74]:
Y=df['kellog_units']
X = df.drop(['kellog_units','mi_price','private_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_3 = sm.OLS(Y,X).fit()
model_3.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [76]:
X = df.drop(['kellog_units','mi_price','private_price'], axis=1)
X = X.select_dtypes(include=['number']).dropna()  # Keep only numeric columns and drop NaNs

Y = df['kellog_units']
Y = Y[X.index]  # Ensure Y is aligned with X

X = sm.add_constant(X)

model_3 = sm.OLS(Y, X).fit()
model_3.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     2563.
Date:                Thu, 23 Jan 2025   Prob (F-statistic):               0.00
Time:                        21:57:20   Log-Likelihood:                -9040.4
No. Observations:               10094   AIC:                         1.809e+04
Df Residuals:                   10090   BIC:                         1.812e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            7.9302      0.074    107.494      0.000       7.786       8.075
kellog_price    -4.1487      0.048    -86.435      0.000      -4.243      -4.055
post_price       0.2728      0.039      7.045      0.000       0.197       0.349
quaker_price     0.2849      0.045      6.323      0.000       0.197       0.373
==============================================================================
Omnibus:                      160.092   Durbin-Watson:                   1.358
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.475
Skew:                          -0.142   Prob(JB):                     1.01e-57
Kurtosis:                       3.737   Cond. No.                         28.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-2.4103/(1-2.4103))
price

2.87284035553134

#### Bias induced by not considering competition

In [81]:
# regression with No competitors
Y=df['kellog_units']
#X=df[['kellog_price']]
X = df.drop(['kellog_units','mi_price','private_price','post_price','quaker_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_1_b = sm.OLS(Y,X).fit()
model_1_b.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [83]:
X = df.drop(['kellog_units','mi_price','private_price','post_price','quaker_price'], axis=1)
X = X.select_dtypes(include=['number']).dropna()  # Keep only numeric columns and drop NaNs

Y = df['kellog_units']
Y = Y[X.index]  # Ensure Y is aligned with X

X = sm.add_constant(X)

model_1_b = sm.OLS(Y, X).fit()
model_1_b.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.427
Method:                 Least Squares   F-statistic:                     7521.
Date:                Thu, 23 Jan 2025   Prob (F-statistic):               0.00
Time:                        21:57:45   Log-Likelihood:                -9088.4
No. Observations:               10094   AIC:                         1.818e+04
Df Residuals:                   10092   BIC:                         1.820e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            8.4460      0.051    166.690      0.000       8.347       8.545
kellog_price    -4.1540      0.048    -86.722      0.000      -4.248      -4.060
==============================================================================
Omnibus:                      150.742   Durbin-Watson:                   1.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.738
Skew:                          -0.141   Prob(JB):                     5.30e-53
Kurtosis:                       3.702   Cond. No.                         17.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [85]:
with_comp = price
#no_comp = (family_price*0.6)*(-4.1540/(1-4.1540))
no_comp = (family_price*0.6)*(-2.4180/(1-2.4180))
no_comp

2.866368111459869

In [87]:
bias = with_comp - no_comp
percent_bias = 100*(bias/with_comp)
percent_bias

0.225290766993351